In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'prod'

In [ ]:
import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore
from typing import Optional
import pandas as pd

In [ ]:
# Define fetch data here using cdutils.database.connect
# There are often fetch_data.py files already in project if migrating
def fetch_data():
    """
    Fetch latest balamt for holdback subaccounts (BALCATCD='HOLD') from ACCTSUBACCT and ACCTBALHIST.
    """
    query = text("""
    SELECT 
        acctnbr,
        subacctnbr,
        balcatcd,
        balamt
    FROM (
        SELECT 
            a.acctnbr,
            a.subacctnbr,
            a.balcatcd,
            h.balamt,
            h.effdate,
            ROW_NUMBER() OVER (PARTITION BY a.acctnbr, a.subacctnbr ORDER BY h.effdate DESC) AS rn
        FROM 
            OSIBANK.ACCTSUBACCT a
        INNER JOIN 
            OSIBANK.ACCTBALHIST h ON a.acctnbr = h.acctnbr AND a.subacctnbr = h.subacctnbr
        WHERE 
            a.BALCATCD = 'HOLD'
    )
    WHERE rn = 1
    """)
    
    queries = [
        {'key':'holdbacks', 'sql':query, 'engine':1},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data


In [ ]:
data = fetch_data()

In [ ]:
df = data['holdbacks'].copy()
df = df[['acctnbr', 'subacctnbr', 'balcatcd', 'balamt']]
assert df[['acctnbr', 'subacctnbr']].is_unique, "Duplicates found in key columns"
assert not df.empty, "No holdback data found"
df.info()

In [ ]:
df.head()